### 네이버 연관 검색어 수집
- 정적(static) 웹페이지 데이터 수집 
- BeautifulSoup을 이용하여 HTML 문자열 데이터 parsing

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### 1. 웹페이지 분석 : URL

In [3]:
query = '삼성전자'
url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={query}'
url

'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=삼성전자'

#### 2. request(URL) > response : str(html)

In [6]:
response = requests.get(url)
response

<Response [200]>

#### 3. str(html) > bs object

In [8]:
dom = BeautifulSoup(response.text, 'html.parser')
type(dom)

bs4.BeautifulSoup

#### 4. bs object > .select(css-selector), .select_one(css-selector) > str(text)

In [28]:
# 10개의 엘리먼트들 선택
# select() : list(Tag, Tag)
# select_one() : Tag
elements = dom.select('.lst_related_srch > .item')

len(elements), elements[0]

(10,
 <li class="item"> <a class="keyword" href="?where=nexearch&amp;query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%EC%A3%BC%EA%B0%80&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0" onclick="return goOtherCR(this, 'a=rsk_top*q.list1&amp;r=1&amp;u=' + urlencode(this.href))"> <div class="tit">삼성전자주가</div> </a> </li>)

#### 5. str(text) > DataFrame

In [33]:
element = elements[0]
keyword = element.select_one('.tit').text
keyword

'삼성전자주가'

In [34]:
keywords = []
for element in elements:
    keyword = element.select_one('.tit').text
    keywords.append(keyword)
keywords

['삼성전자주가',
 '삼성전자 배당금',
 '삼성전자서비스',
 '삼성전자주식',
 '오늘 삼성전자 주가',
 '삼성전자 배당금 지급일',
 '삼성전자몰',
 '삼성 전자레인지',
 '삼성전자 채용',
 '삼성전자 고객센터']

In [36]:
# list comprehension
keywords =[element.select_one('.tit').text for element in elements] 
keywords

['삼성전자주가',
 '삼성전자 배당금',
 '삼성전자서비스',
 '삼성전자주식',
 '오늘 삼성전자 주가',
 '삼성전자 배당금 지급일',
 '삼성전자몰',
 '삼성 전자레인지',
 '삼성전자 채용',
 '삼성전자 고객센터']

In [46]:
# 시간 데이터 넣기
from datetime import datetime
now = datetime.now().strftime('%Y-%m-%d %H:%M') # 현재 시각

In [47]:
df = pd.DataFrame({'keywords' : keywords})
df['query'] = query
df['datetime'] = now
df

,keywords,query,datetime
0,삼성전자주가,삼성전자,2023-08-25 12:21
1,삼성전자 배당금,삼성전자,2023-08-25 12:21
2,삼성전자서비스,삼성전자,2023-08-25 12:21
3,삼성전자주식,삼성전자,2023-08-25 12:21
4,오늘 삼성전자 주가,삼성전자,2023-08-25 12:21
5,삼성전자 배당금 지급일,삼성전자,2023-08-25 12:21
6,삼성전자몰,삼성전자,2023-08-25 12:21
7,삼성 전자레인지,삼성전자,2023-08-25 12:21
8,삼성전자 채용,삼성전자,2023-08-25 12:21
9,삼성전자 고객센터,삼성전자,2023-08-25 12:21


In [48]:
# 함수로 만들기
def naver_relate_keywords(query):

    url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={query}'
    response = requests.get(url)
    dom = BeautifulSoup(response.text, 'html.parser')
    elements = dom.select('.lst_related_srch > .item')
    keywords =[element.select_one('.tit').text for element in elements] 
    df = pd.DataFrame({'keywords' : keywords})
    df['query'] = query
    df['datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M') # 현재 시각
    
    return df

In [60]:
naver_relate_keywords('삼성전자')

,keywords,query,datetime
0,삼성전자주가,삼성전자,2023-08-25 12:24
1,삼성전자 배당금,삼성전자,2023-08-25 12:24
2,삼성전자서비스,삼성전자,2023-08-25 12:24
3,삼성전자주식,삼성전자,2023-08-25 12:24
4,오늘 삼성전자 주가,삼성전자,2023-08-25 12:24
5,삼성전자 배당금 지급일,삼성전자,2023-08-25 12:24
6,삼성전자몰,삼성전자,2023-08-25 12:24
7,삼성 전자레인지,삼성전자,2023-08-25 12:24
8,삼성전자 채용,삼성전자,2023-08-25 12:24
9,삼성전자 고객센터,삼성전자,2023-08-25 12:24


In [61]:
df1 = naver_relate_keywords('삼성전자')
df2 = naver_relate_keywords('LG')

In [63]:
pd.concat([df1, df2], ignore_index=True)

,keywords,query,datetime
0,삼성전자주가,삼성전자,2023-08-25 12:25
1,삼성전자 배당금,삼성전자,2023-08-25 12:25
2,삼성전자서비스,삼성전자,2023-08-25 12:25
3,삼성전자주식,삼성전자,2023-08-25 12:25
4,오늘 삼성전자 주가,삼성전자,2023-08-25 12:25
5,삼성전자 배당금 지급일,삼성전자,2023-08-25 12:25
6,삼성전자몰,삼성전자,2023-08-25 12:25
7,삼성 전자레인지,삼성전자,2023-08-25 12:25
8,삼성전자 채용,삼성전자,2023-08-25 12:25
9,삼성전자 고객센터,삼성전자,2023-08-25 12:25


In [64]:
pd.concat([df1, df2])

,keywords,query,datetime
0,삼성전자주가,삼성전자,2023-08-25 12:25
1,삼성전자 배당금,삼성전자,2023-08-25 12:25
2,삼성전자서비스,삼성전자,2023-08-25 12:25
3,삼성전자주식,삼성전자,2023-08-25 12:25
4,오늘 삼성전자 주가,삼성전자,2023-08-25 12:25
5,삼성전자 배당금 지급일,삼성전자,2023-08-25 12:25
6,삼성전자몰,삼성전자,2023-08-25 12:25
7,삼성 전자레인지,삼성전자,2023-08-25 12:25
8,삼성전자 채용,삼성전자,2023-08-25 12:25
9,삼성전자 고객센터,삼성전자,2023-08-25 12:25
